# Problem Statement

#### Airbnb is one of the biggest online marketplace for arranging or offering lodging, primarily homestays, or tourism experiences. Since this is a vacation time families tend to travel in united states for exploring various places. During this process, they often book their stays through airbnb. So, In this project we tried to analyze the airbnb dataset of newyork for predicting the prices of various apartment listings through different factors and features.

In [3]:
# Importing libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyspark.sql.functions import desc
from pyspark.sql.functions import max
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import VectorAssembler,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import Bucketizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


### Data Cleaning

In [5]:
# Loading the data from ab_nyc_2019 
df = spark.sql("select * from ab_nyc_2019")

In [6]:
#counting the data
df.count()

Out[176]: 49078

In [7]:
# Checking the schema
df.printSchema()


root
-- id: integer (nullable = true)
-- name: string (nullable = true)
-- host_id: integer (nullable = true)
-- host_name: string (nullable = true)
-- neighbourhood_group: string (nullable = true)
-- neighbourhood: string (nullable = true)
-- latitude: float (nullable = true)
-- longitude: float (nullable = true)
-- room_type: string (nullable = true)
-- price: integer (nullable = true)
-- minimum_nights: integer (nullable = true)
-- number_of_reviews: integer (nullable = true)
-- last_review: string (nullable = true)
-- reviews_per_month: float (nullable = true)
-- calculated_host_listings_count: integer (nullable = true)
-- availability_365: integer (nullable = true)

In [8]:
# Findings number of nulls in each column
# reference: https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
 id|name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
183| 199| 351| 372| 351| 351| 351| 351| 351| 351| 351| 351| 10377| 10377| 351| 351|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+

In [9]:
# Dropping nulls in the following columns. Since with nulls in this columns we will not be able to interpret the data
df=df.dropna(subset=['neighbourhood_group','neighbourhood','latitude','longitude','room_type','price','minimum_nights','availability_365','calculated_host_listings_count'])

In [10]:
# Counting the nulls after dropping above columns
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
 id|name|host_id|host_name|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
 0| 16| 0| 21| 0| 0| 0| 0| 0| 0| 0| 0| 10026| 10026| 0| 0|
+---+----+-------+---------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+

In [11]:
# we dont really need last review date , name and host_name for our analysis part since this doesnt affect our price in logical way in real world.
df=df.drop('last_review','name','host_name')

In [12]:
# Rechecking the schema after cleaning the data
df.printSchema()

root
-- id: integer (nullable = true)
-- host_id: integer (nullable = true)
-- neighbourhood_group: string (nullable = true)
-- neighbourhood: string (nullable = true)
-- latitude: float (nullable = true)
-- longitude: float (nullable = true)
-- room_type: string (nullable = true)
-- price: integer (nullable = true)
-- minimum_nights: integer (nullable = true)
-- number_of_reviews: integer (nullable = true)
-- reviews_per_month: float (nullable = true)
-- calculated_host_listings_count: integer (nullable = true)
-- availability_365: integer (nullable = true)

In [13]:
# Counting the data rows
df.count()

Out[183]: 48736

In [14]:
# reviews per month will be zero if the number of reviews are zero ( we can clearly see here the number of reviews and reviews_per_month are of same count when reviews_per_month is zero)
df.select('number_of_reviews','reviews_per_month').filter("reviews_per_month is null").count()

Out[184]: 10029

In [15]:
# replacing the null values of reviews_per_month to zero.
df=df.fillna({ 'reviews_per_month':0 } )

In [16]:
# Final check for any nulls
# reference: https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 id|host_id|neighbourhood_group|neighbourhood|latitude|longitude|room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+---+-------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+-------+-------------------+-------------+--------+---------+---------+-----+--------------+-----------------+-----------------+------------------------------+----------------+

In [17]:
# Final count of the data after cleaning
df.count()

Out[187]: 48736

In [18]:
df.show()

+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 id|host_id|neighbourhood_group| neighbourhood|latitude|longitude| room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
2539| 2787| Brooklyn| Kensington|40.64749|-73.97237| Private room| 149| 1| 9| 0.21| 6| 365|
2595| 2845| Manhattan| Midtown|40.75362|-73.98377|Entire home/apt| 225| 1| 45| 0.38| 2| 355|
3647| 4632| Manhattan| Harlem|40.80902| -73.9419| Private room| 150| 3| 0| 0.0| 1| 365|
3831| 4869| Brooklyn| Clinton Hill|40.68514|-73.95976|Entire home/apt| 89| 1| 270| 4.64| 1| 194|
5022| 7192| Manhattan| East Harlem|40.79851|-73.94399|Entire home/apt| 80| 10| 9| 0.1| 1| 0|
5099| 7322| Manhattan| Murray Hill|40.74767| -73.975|Entire home/apt| 200| 3| 74| 0.59| 1| 129|
5121| 7356| Brooklyn|Bedford-Stuyvesant|40.68688|-73.95596| Private room| 60| 45| 49| 0.4| 1| 0|
5178| 8967| Manhattan| Hell's Kitchen|40.76489|-73.98493| Private room| 79| 2| 430| 3.47| 1| 220|
5203| 7490| Manhattan| Upper West Side|40.80178|-73.96723| Private room| 79| 2| 118| 0.99| 1| 0|
5238| 7549| Manhattan| Chinatown|40.71344|-73.99037|Entire home/apt| 150| 1| 160| 1.33| 4| 188|
5295| 7702| Manhattan| Upper West Side|40.80316|-73.96545|Entire home/apt| 135| 5| 53| 0.43| 1| 6|
5441| 7989| Manhattan| Hell's Kitchen|40.76076|-73.98867| Private room| 85| 2| 188| 1.5| 1| 39|
5803| 9744| Brooklyn| South Slope|40.66829|-73.98779| Private room| 89| 4| 167| 1.34| 3| 314|
6021| 11528| Manhattan| Upper West Side|40.79826|-73.96113| Private room| 85| 2| 113| 0.91| 1| 333|
6090| 11975| Manhattan| West Village| 40.7353|-74.00525|Entire home/apt| 120| 90| 27| 0.22| 1| 0|
6848| 15991| Brooklyn| Williamsburg|40.70837|-73.95352|Entire home/apt| 140| 2| 148| 1.2| 1| 46|
7097| 17571| Brooklyn| Fort Greene|40.69169|-73.97185|Entire home/apt| 215| 2| 198| 1.72| 1| 321|
7322| 18946| Manhattan| Chelsea|40.74192|-73.99501| Private room| 140| 1| 260| 2.12| 1| 12|
7726| 20950| Brooklyn| Crown Heights|40.67592|-73.94694|Entire home/apt| 99| 3| 53| 4.44| 1| 21|
7750| 17985| Manhattan| East Harlem|40.79685|-73.94872|Entire home/apt| 190| 7| 0| 0.0| 2| 249|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
only showing top 20 rows

### Analysis and visualization of the data

In [20]:
# lets find the distinct values in neighbourhood_group
df.select('neighbourhood_group').distinct().show()

+-------------------+
neighbourhood_group|
+-------------------+
 Queens|
 Brooklyn|
 Staten Island|
 Manhattan|
 Bronx|
+-------------------+

In [21]:
# lets find the some of the distinct values in neighbourhood.
df.select('neighbourhood').distinct().show()

+------------------+
 neighbourhood|
+------------------+
 Corona|
 Richmondtown|
 Prince's Bay|
 Westerleigh|
 Mill Basin|
 Civic Center|
 Douglaston|
 Mount Hope|
 Marble Hill|
 Rego Park|
 Dyker Heights|
 Kew Gardens Hills|
 Dongan Hills|
Financial District|
 Bay Terrace|
 Midtown|
 Jackson Heights|
 Bayside|
 Hell's Kitchen|
 Highbridge|
+------------------+
only showing top 20 rows

In [22]:
 # This will give us total number of neighbourhoods.
df.select('neighbourhood').distinct().count()

Out[191]: 221

In [23]:
# lets  find the distinct values in the room type.
df.select('room_type').distinct().show()

+---------------+
 room_type|
+---------------+
 Shared room|
Entire home/apt|
 Private room|
+---------------+

In [24]:
# We will create a temporary view for running spark sql queries to further analyze the data.
df.createOrReplaceTempView('ab_newyork_view')

In [25]:
# Select all the columns from the view
spark.sql("select * from ab_newyork_view").show()

+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
 id|host_id|neighbourhood_group| neighbourhood|latitude|longitude| room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
2539| 2787| Brooklyn| Kensington|40.64749|-73.97237| Private room| 149| 1| 9| 0.21| 6| 365|
2595| 2845| Manhattan| Midtown|40.75362|-73.98377|Entire home/apt| 225| 1| 45| 0.38| 2| 355|
3647| 4632| Manhattan| Harlem|40.80902| -73.9419| Private room| 150| 3| 0| 0.0| 1| 365|
3831| 4869| Brooklyn| Clinton Hill|40.68514|-73.95976|Entire home/apt| 89| 1| 270| 4.64| 1| 194|
5022| 7192| Manhattan| East Harlem|40.79851|-73.94399|Entire home/apt| 80| 10| 9| 0.1| 1| 0|
5099| 7322| Manhattan| Murray Hill|40.74767| -73.975|Entire home/apt| 200| 3| 74| 0.59| 1| 129|
5121| 7356| Brooklyn|Bedford-Stuyvesant|40.68688|-73.95596| Private room| 60| 45| 49| 0.4| 1| 0|
5178| 8967| Manhattan| Hell's Kitchen|40.76489|-73.98493| Private room| 79| 2| 430| 3.47| 1| 220|
5203| 7490| Manhattan| Upper West Side|40.80178|-73.96723| Private room| 79| 2| 118| 0.99| 1| 0|
5238| 7549| Manhattan| Chinatown|40.71344|-73.99037|Entire home/apt| 150| 1| 160| 1.33| 4| 188|
5295| 7702| Manhattan| Upper West Side|40.80316|-73.96545|Entire home/apt| 135| 5| 53| 0.43| 1| 6|
5441| 7989| Manhattan| Hell's Kitchen|40.76076|-73.98867| Private room| 85| 2| 188| 1.5| 1| 39|
5803| 9744| Brooklyn| South Slope|40.66829|-73.98779| Private room| 89| 4| 167| 1.34| 3| 314|
6021| 11528| Manhattan| Upper West Side|40.79826|-73.96113| Private room| 85| 2| 113| 0.91| 1| 333|
6090| 11975| Manhattan| West Village| 40.7353|-74.00525|Entire home/apt| 120| 90| 27| 0.22| 1| 0|
6848| 15991| Brooklyn| Williamsburg|40.70837|-73.95352|Entire home/apt| 140| 2| 148| 1.2| 1| 46|
7097| 17571| Brooklyn| Fort Greene|40.69169|-73.97185|Entire home/apt| 215| 2| 198| 1.72| 1| 321|
7322| 18946| Manhattan| Chelsea|40.74192|-73.99501| Private room| 140| 1| 260| 2.12| 1| 12|
7726| 20950| Brooklyn| Crown Heights|40.67592|-73.94694|Entire home/apt| 99| 3| 53| 4.44| 1| 21|
7750| 17985| Manhattan| East Harlem|40.79685|-73.94872|Entire home/apt| 190| 7| 0| 0.0| 2| 249|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
only showing top 20 rows

In [26]:
# We will count the number of apartments for each neighbourhood group.
display(spark.sql("select count(id) as number_of_apartments,neighbourhood_group from ab_newyork_view group by neighbourhood_group order by number_of_apartments desc"))

number_of_apartments,neighbourhood_group
21595,Manhattan
20056,Brooklyn
5627,Queens
1080,Bronx
369,Staten Island


In [27]:
# Since there are some many neighbourhoods we will find the top ten neighbourhoods with highest number of apartments.
display(spark.sql("select count(id) as number_of_apartments,neighbourhood from ab_newyork_view group by neighbourhood,neighbourhood_group order by number_of_apartments desc limit 10"))

number_of_apartments,neighbourhood
3914,Williamsburg
3705,Bedford-Stuyvesant
2645,Harlem
2461,Bushwick
1968,Upper West Side
1950,Hell's Kitchen
1849,East Village
1790,Upper East Side
1560,Crown Heights
1541,Midtown


In [28]:
#We will count how many apartments are there for each type of apartment
display(spark.sql("select count(id) as number_of_apartments,room_type from ab_newyork_view group by room_type order by number_of_apartments desc"))

number_of_apartments,room_type
25346,Entire home/apt
22225,Private room
1156,Shared room


In [29]:
# We will see what are top ten neighbourhoods with highest apartment count for each apartment type
fig,axes =plt.subplots(1,3,figsize=(15,4))
house_type=['Entire home/apt','Private room','Shared room']
# Using this for loop we will loop through for every apartment type and the count the number of apartments grouped by neighbourhood,room type.
for ax,house in zip(axes,house_type):
  df_agg = spark.sql("select count(id) as number_of_apartments,neighbourhood,room_type from ab_newyork_view   group by neighbourhood,room_type order by number_of_apartments ")
  df_agg=df_agg.filter(df.room_type==house)
  df_agg=df_agg.sort(desc('number_of_apartments')).limit(10)
  pd_df = df_agg.toPandas()  
  g=sns.barplot(x='neighbourhood',y='number_of_apartments',ax=ax,data=pd_df)
  g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
  g.set_title(house)
plt.tight_layout()
display(plt.show())

In [30]:
# We will  count number of apartments for each neighbourhood_group for each apartment type
fig,axes =plt.subplots(1,3,figsize=(15,4))
house_type=['Entire home/apt','Private room','Shared room']
# Using this for loop we will loop through for every apartment type and the count the number of apartments grouped by neighbourhood_group,room type.
for ax,house in zip(axes,house_type):
  df_agg = spark.sql("select count(id) as number_of_apartments,neighbourhood_group,room_type from ab_newyork_view   group by neighbourhood_group,room_type order by number_of_apartments ")
  df_agg=df_agg.filter(df.room_type==house)
  df_agg=df_agg.sort(desc('number_of_apartments')).limit(10)
  pd_df = df_agg.toPandas()  
  g=sns.barplot(x='neighbourhood_group',y='number_of_apartments',ax=ax,data=pd_df)
  g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
  g.set_title(house)
plt.tight_layout()
display(plt.show())

In [31]:
#We will see what are the top ten neighbourhoods with highest averge price for each apartment type
fig,axes =plt.subplots(1,3,figsize=(10,4))
house_type=['Entire home/apt','Private room','Shared room']
# Using this for loop we will loop through for every apartment type and calculate the number of average price grouped by neighbourhood,room type.
for ax,house in zip(axes,house_type):
  df_agg = spark.sql("select avg(price) as avg_price,neighbourhood,room_type from ab_newyork_view   group by neighbourhood,room_type order by avg_price ")
  df_agg=df_agg.filter(df.room_type==house)
  df_agg=df_agg.sort(desc('avg_price')).limit(10)
  pd_df = df_agg.toPandas()  
  g=sns.barplot(x='neighbourhood',y='avg_price',ax=ax,data=pd_df)
  g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
  g.set_title(house)
plt.tight_layout()
display(plt.show())

In [32]:
#We will calculate the avg price of each apartment type in every neighbourhood_group
fig, axes =plt.subplots(1,3, figsize=(10,4))
house_type=['Entire home/apt','Private room','Shared room']
# Using this for loop we will loop through for every apartment type and calculate the number of average price grouped by neighbourhood_group,room type.
for ax,house in zip(axes,house_type):
  df_agg = spark.sql("select avg(price) as avg_price,neighbourhood_group,room_type from ab_newyork_view   group by neighbourhood_group,room_type order by avg_price ")
  df_agg=df_agg.filter(df.room_type==house)
  df_agg=df_agg.sort(desc('avg_price')).limit(10)
  pd_df = df_agg.toPandas()
  g=sns.barplot(x='neighbourhood_group',y='avg_price',ax=ax,data=pd_df)
  g.set_xticklabels(g.get_xticklabels(), rotation=40,ha="right")
  g.set_title(house)
plt.tight_layout()
display(plt.show())

In [33]:
#  we can also observe that though williamsburg has the highest number of apartments Bedford-Stuyvesant is highely popular amongst the people based on the number of reviews.
display(spark.sql("select sum(number_of_reviews) as Reviews_Count,neighbourhood from ab_newyork_view group by\
          neighbourhood order by Reviews_Count desc limit 10"))

Reviews_Count,neighbourhood
110194,Bedford-Stuyvesant
85368,Williamsburg
75764,Harlem
52404,Bushwick
49729,Hell's Kitchen
44605,East Village
36393,Crown Heights
36167,East Harlem
36049,Upper West Side
31581,Upper East Side


In [34]:
display(spark.sql("select sum(number_of_reviews) as Reviews_Count,neighbourhood_group from ab_newyork_view group by\
          neighbourhood_group order by Reviews_Count desc limit 10"))

Reviews_Count,neighbourhood_group
485733,Brooklyn
452560,Manhattan
155751,Queens
27975,Bronx
11393,Staten Island


In [35]:
# we can also see that People mostly prefer Entire home/apt and Private room for renting 
display(spark.sql("select sum(number_of_reviews) as Reviews_Count,room_type from ab_newyork_view group by\
          room_type order by Reviews_Count desc limit 10"))

Reviews_Count,room_type
578258,Entire home/apt
536006,Private room
19148,Shared room


In [36]:
# As we can see top three apartment listings are for minimum 1-3 nights. Also we can see there are significant apartment listings for 30 minimum nights
display(spark.sql("select count(id) as number_of_apartments,minimum_nights from ab_newyork_view group by\
          minimum_nights order by number_of_apartments desc limit 10"))

number_of_apartments,minimum_nights
12662,1
11650,2
7969,3
3755,30
3299,4
3027,5
2052,7
748,6
560,14
483,10


In [37]:
# Convert the spark dataframe to pandas dataframe for better visualization
pd_df1=df.toPandas()
print(pd_df1.columns)

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
Index(['id', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
 'longitude', 'room_type', 'price', 'minimum_nights',
 'number_of_reviews', 'reviews_per_month',
 'calculated_host_listings_count', 'availability_365'],
 dtype='object')

In [38]:
# We will see if there is any relation between number of reviews and prices.Through this we can see the apartments of which price range are popular amongst the people.
# We can see that most reviews are given for the apartments with price less than 500. So apartments below 500 price are very popular amongst people.
fig, ax = plt.subplots( figsize=(10,4))
g=sns.scatterplot(pd_df1['price'],pd_df1['number_of_reviews'])
g.set_xticks(np.arange(0,10000,500))
plt.tight_layout()
display(plt.show())

In [39]:
# we will create a map with neighbourhood groups to easily understand the newyork city
fig, ax = plt.subplots()
g=sns.scatterplot(pd_df1['longitude'],pd_df1['latitude'],hue=pd_df1['neighbourhood_group'])
display(plt.show())

In [40]:
# We will see what are the most busiest neighbourhood groups in terms of apartments availability. We can here see that manhattan and brooklyn neighbourhoods has the apartments with most number of zero availabilty.
fig, ax = plt.subplots()
g=sns.scatterplot(pd_df1['longitude'],pd_df1['latitude'],hue=pd_df1['availability_365'])
display(plt.show())

In [41]:
# we can also see the distribution of availability for each neighbourhood_group through box plot.
fig =plt.subplots(figsize=(8,4))
ax=sns.boxplot(pd_df1['neighbourhood_group'],pd_df1['availability_365'])
display(ax)

In [42]:
# We will see if there is any relation between reviews and availability. Ideally if a listing has highest reviews per month there it is very likely to  be less available. 
# But as we can see in the graph there is no such indication.
fig, ax =plt.subplots(figsize=(8,4), sharex=True)
ax=sns.scatterplot(pd_df1['availability_365'],pd_df1['reviews_per_month'])
display(ax)

In [43]:
# We can also see the price distribution. we can see most of the apartments are below price 1000
fig, ax =plt.subplots(figsize=(15,4), sharex=True)
ax=sns.distplot(pd_df1['price'],kde=False)
ax.set_xticks(np.arange(0,10000,step=500))
#ax.set_xticklabels(ax.get_xticklabels(), rotation=40,ha="right")
plt.tight_layout()
display(fig)

In [44]:
fig =plt.subplots(figsize=(8,4))
ax=sns.boxplot(pd_df1['price'])
display(ax)

In [45]:
# We will see if host id has any connection with the price. But there seems to be no connection
fig, ax =plt.subplots(figsize=(8,4))
ax=sns.scatterplot(pd_df1['host_id'],pd_df1['price'])
#ax.set_xticks(np.arange(0,10000,step=1000))
#ax.set_xticklabels(ax.get_xticklabels(), rotation=40,ha="right")
plt.tight_layout()
display(fig)

### Finding correlation matrix

In [48]:
# We will using Bucketizer try to find the range count of the price to determine if there are any outliers
df_price=df.select('price')
splits = [0.0, 1000.0, 3000.0, 5000.0, 7000.0,float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="price", outputCol="bucketedFeatures")
bucketedData = bucketizer.transform(df_price)
bucketedData.groupBy("bucketedFeatures").count().show()

+----------------+-----+
bucketedFeatures|count|
+----------------+-----+
 0.0|48439|
 1.0| 245|
 4.0| 11|
 3.0| 14|
 2.0| 27|
+----------------+-----+

In [49]:
# From the above result we found that our most of our data lies between price range 0 and 1000
df=df.filter('price <=1000')

In [50]:
# Indexing categorical variables neighbourhood , neighbourhood_indexer and room type
neighbourhood_indexer = StringIndexer(inputCol='neighbourhood',outputCol='neigbourhood_index',handleInvalid='keep')
neighbourhood_group_indexer = StringIndexer(inputCol='neighbourhood_group',outputCol='neighbourhood_group_index',handleInvalid='keep')
room_type_indexer = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')

In [51]:
# Create a vector assembler with all input features and indexed features
assembler_corr = VectorAssembler(inputCols=['neigbourhood_index','neighbourhood_group_index','latitude','longitude',
                                       'room_type_index','minimum_nights','number_of_reviews','price',
                                       'calculated_host_listings_count','availability_365'],
                            outputCol="features")

In [52]:
# create a pipeline with stages of transformation
pipe = Pipeline(stages=[neighbourhood_indexer,neighbourhood_group_indexer,room_type_indexer,assembler_corr])

In [53]:
# fit the pipeline to our spark dataframe df
fit_pipe = pipe.fit(df)

In [54]:
# create a transformed data frame 
df_correlated=fit_pipe.transform(df)
# Getting the correlation dense matrix
corr = Correlation.corr(df_correlated, "features").collect()[0][0]
# Coverting it to a list so that it can plotted
corrmatrix = corr.toArray().tolist() 
# defining the features data columns since if convert the corr matrix to list we will loose column headers
data_columns=[
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'minimum_nights',
 'number_of_reviews',
   'price',
 'calculated_host_listings_count',
 'availability_365']
# We create a dataframe with correlated values list between features and features headers
corrdf=spark.createDataFrame(corrmatrix,data_columns)

In [55]:
# We will plot the correlation matrix with matplot matrix show.
# reference: https://stackoverflow.com/questions/55546467/how-to-plot-correlation-heatmap-when-using-pysparkdatabricks
fig, ax = plt.subplots()
# Creating a matrix plot with correlation matrix with max value 1 and min value -1
corr_ax=ax.matshow(corrmatrix,vmax=1,vmin=-1)
# Setting title xlabel , ylabel, xticks and yticks
ax.set_title("Correlation Matrix")
ax.set_xticks(np.arange(len(data_columns)))
ax.set_yticks(np.arange(len(data_columns)))
ax.set_xticklabels(data_columns)
ax.set_yticklabels(data_columns)
plt.gca().xaxis.tick_bottom()
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.tick_params(axis="both", which="both" )
fig.colorbar(corr_ax)
# Adding correlation value  using every element of corr matrix on the matrix plot 
for i in range(len(corrmatrix)):
  for j in range(len(corrmatrix[0])):
          text = ax.text(j, i, "%.2f" % corrmatrix[i][j],
                         ha="center", va="center", color="white", fontsize="x-small")
fig.tight_layout()
display(fig)

## Modelling for predicting the price of an apartment listing

### Linear Regression

In [58]:
# For our model building we will consider two categorical columns. Neighbourhood group and room type. We are not taking neighbourhood categorical column since we can determine the location with the help of latitude and longitude
neighbourhood_group_indexer_lr = StringIndexer(inputCol='neighbourhood_group',outputCol='neighbourhood_group_index_lr',handleInvalid='keep')
room_type_indexer_lr = StringIndexer(inputCol='room_type',outputCol='room_type_index_lr',handleInvalid='keep')

In [59]:
# Create a input features vector
assembler_lr = VectorAssembler(inputCols=['neighbourhood_group_index_lr','latitude','longitude',
                                       'room_type_index_lr','minimum_nights','number_of_reviews',
                                       'calculated_host_listings_count','availability_365'],
                            outputCol="features")

In [60]:
# Creating a pipeline object with all the stages
pipe_lr = Pipeline(stages=[neighbourhood_group_indexer_lr,room_type_indexer_lr,assembler_lr])

In [61]:
# Splitting the data to get train and test data
train_lr,test_lr=df.randomSplit([0.7,0.3])

In [62]:
# Fitting the training data
fit_pipe_lr=pipe_lr.fit(train_lr)
train_lr=fit_pipe_lr.transform(train_lr)

In [63]:
# Creating a Linear Regression model object
lr_model = LinearRegression(labelCol='price')

In [64]:
# Training the model with train data
fit_model_lr = lr_model.fit(train_lr.select(['features','price']))
test_lr=fit_pipe_lr.transform(test_lr)
lr_results = fit_model_lr.transform(test_lr)

In [65]:
lr_results.select(['price','prediction']).show()

+-----+------------------+
price| prediction|
+-----+------------------+
 200| 204.2667567443168|
 79| 57.89196366176475|
 150|198.49483967695414|
 89| 99.91284795388492|
 120|172.50541245324348|
 215|182.05108228223253|
 299|169.62509424245218|
 130|111.14226169107496|
 80|133.40166010356916|
 120|181.32391544700658|
 50|111.24836846605831|
 89|103.70588556917573|
 85|116.40007838590827|
 150|133.76739824585457|
 120|126.34964047697213|
 150|220.68323563477315|
 130|40.168471879293065|
 110|194.74863288647248|
 80| 88.62678546294774|
 151|232.47656462610757|
+-----+------------------+
only showing top 20 rows

In [66]:
# Lets  evaluate the test results that we got by the model
test_results_lr = fit_model_lr.evaluate(test_lr)

In [67]:
# Finding the residuals
test_results_lr.residuals.show()

+-------------------+
 residuals|
+-------------------+
 -4.266756744316808|
 21.10803633823525|
 -48.49483967695414|
 -10.91284795388492|
 -52.50541245324348|
 32.94891771776747|
 129.37490575754782|
 18.857738308925036|
 -53.40166010356916|
-61.323915447006584|
 -61.24836846605831|
-14.705885569175734|
-31.400078385908273|
 16.232601754145435|
 -6.349640476972127|
 -70.68323563477315|
 89.83152812070693|
 -84.74863288647248|
 -8.626785462947737|
 -81.47656462610757|
+-------------------+
only showing top 20 rows

In [68]:
# The root mean square error is as follows
test_results_lr.rootMeanSquaredError

Out[233]: 96.14557149921103

In [69]:
# R2 score of the model.
test_results_lr.r2

Out[234]: 0.30005264244375696

In [70]:
# Lets convert the results data frame to pandas dataframe for plotting the actual price vs predicted price 
pd_df_lr = lr_results.toPandas()

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [71]:
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.regplot(pd_df_lr['prediction'],pd_df_lr['price'])
display(plt.show())

### Random Forest Regressor

In [73]:
# For our model building we will consider two categorical columns. Neighbourhood group and room type. We are not taking neighbourhood categorical column since we can determine the location with the help of latitude and longitude
neighbourhood_group_indexer_rf = StringIndexer(inputCol='neighbourhood_group',outputCol='neighbourhood_group_index',handleInvalid='keep')
room_type_indexer_rf = StringIndexer(inputCol='room_type',outputCol='room_type_index',handleInvalid='keep')

In [74]:
# Creating a vector with input features
assembler_rf = VectorAssembler(inputCols=['neighbourhood_group_index','latitude','longitude',
                                       'room_type_index','minimum_nights','number_of_reviews',
                                       'calculated_host_listings_count','availability_365'],
                            outputCol="features")

In [75]:
# Splitting the train and test dataset.
train_rf,test_rf=df.randomSplit([0.7,0.3])

In [76]:
# Creating Random Forest Regressor model object
rf_model=RandomForestRegressor(featuresCol="features",labelCol='price')

In [77]:
#Creating the pipeline object with Pipeline stages for transforming the data and applying model
pipe_rf = Pipeline(stages=[neighbourhood_group_indexer_rf,room_type_indexer_rf,assembler_rf,rf_model])


In [78]:
# Fitting the training data to model
rf_model = pipe_rf.fit(train_rf)


In [79]:
# Testing the model with test dataset
result_rf = rf_model.transform(test_rf)


In [80]:
# Printing our price and prediction columns of result
result_rf.select(['price','prediction']).show()


+-----+------------------+
price| prediction|
+-----+------------------+
 80|164.90653182935847|
 60| 68.3637451087704|
 85|127.32643234018217|
 89| 84.49414512077574|
 140|127.32643234018217|
 99|146.67594364603679|
 190|203.00250497219378|
 80|103.91537450224027|
 120|160.93338653668548|
 55| 79.28169863333723|
 35| 83.8679232768558|
 150|187.67339280307974|
 130| 81.5765595915619|
 80| 71.96706836548792|
 151| 262.343785793592|
 144|191.10267591076246|
 250| 190.0213792385503|
 52|125.52533335581636|
 225|175.29940427516584|
 80|122.68767602268088|
+-----+------------------+
only showing top 20 rows

In [81]:
# Evaluating Decision Tree regression model based on the Regression Evaluator.
evaluate = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="rmse")

In [82]:
# Calculating the root mean square value for the model.
rmse = evaluate.evaluate(result_rf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 92.2806

In [83]:
# Creating a another Regression Evaluator object for calculating r2 score.
evaluate1 = RegressionEvaluator(
    labelCol="price", predictionCol="prediction", metricName="r2")


In [84]:
r2 = evaluate1.evaluate(result_rf)
print("R Squared (R2) on test data = %g" % r2)

R Squared (R2) on test data = 0.362342

In [85]:
# Creating pandas datafram for result set
pd_df_rf = result_rf.toPandas()

/databricks/spark/python/pyspark/sql/dataframe.py:2153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)

In [86]:
# Plotting the actual price and predicted price
fig, ax=plt.subplots(figsize=(10,6))
ax=sns.regplot(pd_df_rf['prediction'],pd_df_rf['price'])
display(plt.show())

# Conclusions

d ### Model Interpretation 
#### Based on the above two models, the prediction of price by the two models seem to be reasonalble, though not good. The analysis of the model and dataset gives us the following interpretations.
1) The correlation matrix suggests the maximum correlation for price is with room type column which is around 0.46. The fact that the two models have R2 value of 0.29 and 0.35 each suggest the models have a reasonable performance.

2)To predict the price more accurately, we would need more features that have more influence on the price and are critical to price of the listings.

3)The Airbnb dataset analysis provided us with finding interesting relations between various features which influence the booking  of a room via Airbnb. The price feature having the most correlation with room type makes sense because  the price will be higher for a Entire home/apt as compared to the shared room.

### Data Analysis
1) There are five distinct neighbourhood groups in newyork city 

2) There are total 221 neighbourhoods in which the apartments have been listed

3) The apartments listed basically are of three types: Entire home/apt, Private room, Shared room

4) Manhattan Neighbourhood group has got the highest number of apartments listed with Staten Island neighbourhood group being the least.

5) Williamsburg has got the highest number of apartments listed followed by Bedford-Stuyvesant

6) Entire home/apt is the highest number of apartment type that are being listed with Shared room type being the least.

7) In neighbourhoods Williamsburg has got the highest number of Entire home/apt type apartments listed, Bedford-Stuyvesant has got the highest number of Private room type apartments listed and Hell's Kitchen has got the highest number of Shared room apartments listed.

8) In neighbourhood groups Manhattan has got the highest number of Entire home/apt type apartments listed, Brooklyn has got the highest number of Private room type apartments listed and Manhattan has got the highest number of Shared room apartments listed.

9) In neighbourhood_groups Manhattan has the highest average price for Entire home/apt,Private room and Shared room apartment types

10) In neighbourhoods Fort Wadsworth has the highest average price for Entire home/apt, Rivedale for private room type and Shared room type.

11) Bedford-Stuyvesant has got highest number of reviews indicating that it is the most popular neighbourhoods.

12) Also Based on number of reviews we can say that people mostly prefer Entire home/apt or a Private room.

13) Most of the apartments are listed for minimum one night. Surprisingly there are many number of apartments listed for a minimum of 30 days.

14) The relationship between number of reviews and prices indicate that people mostly prefer apartments for the price below $1000.

15) Practically if the reviews per month are highest the availability 365 should be high. But there seems to be no connection.

16) In price distribution we can see that mostly the prices are below 1000$. There are also some outliers.

17) From the box plot we can understand that brooklyn has low average availability in terms of number of days for which a apartment is available. This is understandable because brooklyn is highly popular for apartments based on number of reviews. Also staten island has high average availability in terms of number of days for which a apartment is available. This is because staten island is not so popular for its apartments based on number of reviews.

18) Also There seems to be no relation between the host id and price.